# Forecasting Demo

This notebook demonstrates the three modeling modules in `timeseries_toolkit`:

1. **Kalman Filter** - State-space modeling with `AutoKalmanFilter`
2. **Regime Detection** - HMM-based market regime identification
3. **Global Boosting Forecaster** - LightGBM multi-entity forecasting

All examples use real market data.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

print('Setup complete.')

Setup complete.


## 1. Fetch Data

In [2]:
from timeseries_toolkit.data_sources import CryptoDataLoader, EquityDataLoader

crypto = CryptoDataLoader()
btc_df = crypto.get_prices(['BTC-USD'], period='2y')
btc_close = btc_df[[c for c in btc_df.columns if 'Close' in c or 'close' in c][0]]
btc_close.name = 'BTC-USD'

equities = EquityDataLoader()
spy_df = equities.get_prices(['SPY'], period='2y')
spy_close = spy_df[[c for c in spy_df.columns if 'Close' in c or 'close' in c][0]]
spy_close.name = 'SPY'

print(f'BTC-USD: {len(btc_close)} days, last price: ${btc_close.iloc[-1]:,.0f}')
print(f'SPY:     {len(spy_close)} days, last price: ${spy_close.iloc[-1]:,.2f}')

BTC-USD: 732 days, last price: $87,825
SPY:     502 days, last price: $695.42


---
## 2. Kalman Filter

The `AutoKalmanFilter` wraps `statsmodels.UnobservedComponents` and provides:
- Automatic model fitting
- Smoothing (extract the latent trend from noisy observations)
- Multi-step forecasting
- Component extraction (trend, cycle, seasonal)

### 2.1 Fit and Smooth SPY

In [3]:
from timeseries_toolkit.models import AutoKalmanFilter

# Prepare daily series with frequency
spy_daily = spy_close.copy()
spy_daily = spy_daily.asfreq('D', method='ffill')

kf = AutoKalmanFilter(level='local linear trend')
kf.fit(spy_daily)

smoothed = kf.smooth()
print(f'Smoothed series length: {len(smoothed)}')
print(f'Last smoothed value: ${smoothed.iloc[-1]:,.2f}')
print(f'Last actual value:   ${spy_daily.iloc[-1]:,.2f}')
print(f'Smoothing error:     ${abs(smoothed.iloc[-1] - spy_daily.iloc[-1]):,.2f}')

Smoothed series length: 731
Last smoothed value: $695.43
Last actual value:   $695.42
Smoothing error:     $0.01


### 2.2 Forecast SPY

In [4]:
forecast = kf.forecast(steps=7)

print('SPY 7-Day Kalman Forecast:')
for i, (date, val) in enumerate(forecast.items(), 1):
    print(f'  Day {i}: ${val:,.2f}')

print(f'\nForecast direction: {"UP" if forecast.iloc[-1] > spy_daily.iloc[-1] else "DOWN"}')
pct_change = (forecast.iloc[-1] / spy_daily.iloc[-1] - 1) * 100
print(f'Expected 7-day change: {pct_change:+.2f}%')

SPY 7-Day Kalman Forecast:
  Day 1: $695.73
  Day 2: $696.02
  Day 3: $696.32
  Day 4: $696.62
  Day 5: $696.91
  Day 6: $697.21
  Day 7: $697.50

Forecast direction: UP
Expected 7-day change: +0.30%


### 2.3 Extract Components

In [5]:
components = kf.get_components()

print('Kalman filter components:')
for name, comp in components.items():
    if comp is not None and len(comp) > 0:
        print(f'  {name}: length={len(comp)}, last value={comp.iloc[-1]:.4f}')

Kalman filter components:
  level: length=731, last value=695.4324
  trend: length=731, last value=0.2960


### 2.4 Kalman vs ARIMA Comparison

In [6]:
from timeseries_toolkit.models import compare_kalman_vs_arima

# Use last 200 days of SPY, holdout last 7 for testing
spy_sample = spy_daily.dropna().tail(200)

comparison = compare_kalman_vs_arima(spy_sample, holdout=7)

print('Kalman vs ARIMA Benchmark (7-day holdout):')
print(f'  Kalman RMSE: {comparison["kalman_rmse"]:.4f}')
print(f'  ARIMA RMSE:  {comparison["arima_rmse"]:.4f}')
print(f'  Kalman MAE:  {comparison["kalman_mae"]:.4f}')
print(f'  ARIMA MAE:   {comparison["arima_mae"]:.4f}')
print(f'  Winner:      {comparison["winner"]}')
print(f'  ARIMA order: {comparison["arima_order"]}')

Kalman vs ARIMA Benchmark (7-day holdout):
  Kalman RMSE: 5.1988
  ARIMA RMSE:  6.6843
  Kalman MAE:  4.7116
  ARIMA MAE:   6.0728
  Winner:      kalman
  ARIMA order: (0, 1, 0)


---
## 3. Regime Detection (HMM)

The `RegimeDetector` uses a Gaussian Mixture HMM (GMMHMM) to identify latent market regimes. It:
- Automatically selects the optimal number of states via BIC
- Returns the Viterbi path (most likely regime sequence)
- Computes smoothed probabilities and transition matrices

### 3.1 Detect Regimes in BTC

In [7]:
from timeseries_toolkit.models import RegimeDetector

# Compute log returns for regime detection
btc_returns = np.log(btc_close / btc_close.shift(1)).dropna()

detector = RegimeDetector(max_states=4)
detector.fit(btc_returns, auto_select=True)

regimes = detector.predict_regimes()
n_states = regimes.nunique()

print(f'Optimal number of states: {n_states}')
print(f'Current regime: {regimes.iloc[-1]}')
print(f'\nRegime distribution:')
regime_counts = regimes.value_counts().sort_index()
for state, count in regime_counts.items():
    pct = count / len(regimes) * 100
    print(f'  State {state}: {count} days ({pct:.1f}%)')

Optimal number of states: 2
Current regime: 0

Regime distribution:
  State 0: 528 days (72.2%)
  State 1: 203 days (27.8%)


### 3.2 Regime Statistics

In [8]:
stats = detector.get_regime_statistics()
print('Regime Statistics:')
stats

Regime Statistics:


,mean,std,min,max,count,proportion
regime,,,,,,
0,0.000692,0.014532,-0.040063,0.041879,528,0.722298
1,0.001684,0.040474,-0.090823,0.114616,203,0.277702


### 3.3 Transition Matrix

In [9]:
trans = detector.get_transition_matrix()
print('Regime Transition Matrix:')
print('(Rows = from state, Columns = to state)')
trans

Regime Transition Matrix:
(Rows = from state, Columns = to state)


,to_regime_0,to_regime_1
from_regime_0,0.820157,0.179843
from_regime_1,0.289796,0.710204


### 3.4 Regime Probabilities

In [10]:
try:
    probs = detector.get_regime_probabilities()
    print(f'Regime probabilities shape: {probs.shape}')
    print(f'\nLast 5 days probabilities:')
    print(probs.tail())
except Exception as e:
    print(f'Probabilities not available: {e}')
    print('(This can happen when the HMM uses fewer states than requested.)')

Regime probabilities shape: (731, 2)

Last 5 days probabilities:
            regime_0  regime_1
Date                          
2026-01-25  0.663127  0.336873
2026-01-26  0.756298  0.243702
2026-01-27  0.852193  0.147807
2026-01-28  0.873554  0.126446
2026-01-29  0.808917  0.191083


### 3.5 Detect Regimes in SPY

In [11]:
spy_returns = np.log(spy_close / spy_close.shift(1)).dropna()

spy_detector = RegimeDetector(max_states=4)
spy_detector.fit(spy_returns, auto_select=True)
spy_regimes = spy_detector.predict_regimes()

print(f'SPY optimal states: {spy_regimes.nunique()}')
print(f'Current regime: {spy_regimes.iloc[-1]}')

spy_stats = spy_detector.get_regime_statistics()
print('\nSPY Regime Statistics:')
spy_stats

Model is not converging.  Current: 1099.335677901674 is not greater than 1113.1403553943471. Delta is -13.804677492673136


Covariance of state #3, mixture #0 has a null eigenvalue.


Model is not converging.  Current: 1405.5675428307356 is not greater than 1418.3636899467053. Delta is -12.796147115969688


Covariance of state #3, mixture #0 has a null eigenvalue.


SPY optimal states: 3
Current regime: 0

SPY Regime Statistics:


,mean,std,min,max,count,proportion
regime,,,,,,
0,0.001973,0.004749,-0.008999,0.012426,375,0.748503
1,-0.003024,0.013778,-0.030257,0.032513,111,0.221557
2,-0.002140,0.038904,-0.060327,0.099863,15,0.029940


---
## 4. Global Boosting Forecaster (LightGBM)

The `GlobalBoostForecaster` trains a single LightGBM model across multiple entities (countries, assets) with mixed-frequency feature support.

### 4.1 Prepare Multi-Entity Data

In [12]:
from timeseries_toolkit.models import GlobalBoostForecaster

# Create a simple multi-entity dataset from market data
# Entity 1: BTC returns with lagged features
# Entity 2: SPY returns with lagged features

def make_entity(series, name):
    """Create entity dict with lagged features."""
    returns = series.pct_change().dropna()
    features = {
        f'{name}_lag1': returns.shift(1),
        f'{name}_lag2': returns.shift(2),
        f'{name}_lag5': returns.shift(5),
        f'{name}_vol5': returns.rolling(5).std(),
    }
    # Drop NaN rows
    valid_idx = returns.index[5:]  # after lags settle
    y = returns.loc[valid_idx]
    X = {k: v.loc[valid_idx] for k, v in features.items()}
    return {'y': y, 'X': X}

all_data = {
    'BTC': make_entity(btc_close, 'btc'),
    'SPY': make_entity(spy_close, 'spy'),
}

print(f'BTC training samples: {len(all_data["BTC"]["y"])}')
print(f'SPY training samples: {len(all_data["SPY"]["y"])}')

BTC training samples: 726
SPY training samples: 496


### 4.2 Train and Predict

In [13]:
forecaster = GlobalBoostForecaster(random_state=42)
forecaster.fit(all_data)

# Predict the last known point for each entity
for entity_name, entity_data in all_data.items():
    # predict() needs the full series for feature engineering;
    # n_periods=1 returns only the last prediction.
    pred = forecaster.predict(
        entity_data['X'], y_series=entity_data['y'],
        entity_id=entity_name, n_periods=1
    )
    actual = entity_data['y'].iloc[-1]
    print(f'{entity_name}: predicted return = {pred[0]:.6f}, actual = {actual:.6f}')

print('\nGlobal model trained successfully across both entities.')

BTC: predicted return = -0.000508, actual = -0.015240
SPY: predicted return = 0.002054, actual = -0.000101

Global model trained successfully across both entities.


---
## Summary

| Model | Class | Strengths |
|-------|-------|----------|
| Kalman Filter | `AutoKalmanFilter` | Handles non-stationarity natively, probabilistic framework, component decomposition |
| HMM Regime Detection | `RegimeDetector` | Identifies market states, provides transition probabilities |
| LightGBM Forecaster | `GlobalBoostForecaster` | Multi-entity training, feature-rich, fast |

The MarketIntelligence system (Notebook 04) automatically selects the best model based on data characteristics.